In [1]:
!pip install imgaug
!pip install keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [ ]:
import math
import pandas as pd
import os
from os import path
from tqdm import tqdm
import json
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import random
from PIL import Image
import pickle
import joblib
import re

from sklearn import preprocessing
import tensorflow as tf
from keras.layers import Input,Dense,LSTM,Flatten,Dropout,concatenate,Conv1D,MaxPooling2D,Activation
from keras.layers import BatchNormalization
from keras.layers import Embedding
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import keras_tuner
from tensorflow import keras
import tensorflow_hub as hub
import imgaug.augmenters as iaa
from tensorflow.keras.preprocessing import image, text, sequence
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import plot_model


import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 1. Modelling on MSCOCO Dataset

### 1.1 Read Data

In [3]:
preprocessed_trainData = pd.read_csv("/kaggle/input/vqa-mscoco/mscoco_train2014_preprocessed.csv")
print("Number of Datapoints in MSCOCO Dataset:",len(preprocessed_trainData))
preprocessed_trainData.head(3)

Number of Datapoints in MSCOCO Dataset: 443757


,Unnamed: 0,image_id,question_id,question_type,answers,answer_type,processed_questions,processed_annotations
0,0,train2014/COCO_train2014_000000458752.jpg,458752000,what is this,"['net', 'net', 'net', 'netting', 'net', 'net',...",other,what is this photo taken looking through,net
1,1,train2014/COCO_train2014_000000458752.jpg,458752001,what,"['pitcher', 'catcher', 'pitcher', 'pitcher', '...",other,what position is this man playing,pitcher
2,2,train2014/COCO_train2014_000000458752.jpg,458752002,what color is the,"['orange', 'orange', 'orange', 'orange', 'oran...",other,what color is the players shirt,orange


In [4]:
print("Unique number of answers of MSCOCO Dataset:",len(set(preprocessed_trainData['processed_annotations'])))

Unique number of answers of MSCOCO Dataset: 22350


I will be considering the top 1000 answers from the answer_df, which I will consider as class labels

In [5]:
# # create dataframe of unique answers and its counts in decending order of answer_count
# processed_annotations = list(preprocessed_trainData['processed_annotations'])
# count = {}
# for i in processed_annotations:
#     count[i] = count.get(i, 0) + 1
    
# answer_df = pd.DataFrame(list(count.items()),columns=["answer","answer_count"])
# answer_df["answer%"] = answer_df["answer_count"]/len(processed_annotations)*100  
# answer_df = answer_df.sort_values(by='answer_count',ascending=False)
# answer_df.head(5)

processed_annotations = list(preprocessed_trainData['processed_annotations'])
count = {}
for i in processed_annotations:
    count[i] = count.get(i, 0) + 1

answer_df = pd.DataFrame(list(count.items()), columns=["answer", "answer_count"])
answer_df["answer%"] = answer_df["answer_count"] / len(processed_annotations) * 100

# Filter out rows where answer_count is less than or equal to 4
answer_df = answer_df[answer_df['answer_count'] > 4]

answer_df = answer_df.sort_values(by='answer_count', ascending=False)
answer_df.head(5)

,answer,answer_count,answer%
3,yes,84978,19.149670
11,no,82516,18.594862
15,1,12541,2.826096
20,2,12215,2.752633
4,white,8916,2.009208


In [6]:
answer_df.shape

(3816, 3)

Exported the answer_df to csv file and eliminated the words with count less than 4 and reading the file again. Now we will be having the labels = 3400 labels.

In [7]:
# answer_df = pd.read_csv("/kaggle/input/vqa-mscoco/answer_df.csv", encoding='utf-8')
# answer_df.head()

In [8]:
# answer_df.shape

In [9]:
# answer_df = answer_df.dropna()

In [10]:
# answer_df.shape

In [ ]:
# Consider top 3816 answers as class label
top_3816_answers = list(answer_df['answer'])[:3816]

data_df = pd.DataFrame()
for i in (range(len(top_3816_answers))):
    # For each index "i", I select the preprocessed_trainData where the processed_annotations matches the i-th top answer column, then I concatenate with the data_df
    data_df = pd.concat([data_df, preprocessed_trainData[preprocessed_trainData.processed_annotations == top_3816_answers[i]]])

print(f"Top 3816 answers coverd {round(len(data_df)/443757*100,2)}% of datapoints")

In [10]:
data_df.shape

(417568, 8)

In [11]:
# Execute these cell once
labelencoder_3816 = preprocessing.LabelEncoder()
labelencoder_3816.fit(top_3816_answers)
pickle.dump((labelencoder_3816),open('/kaggle/working/labelencoder_3816.pkl','wb'))

In [ ]:
# print(data_df.columns)


In [12]:
# labelencoder = pickle.load(open('/kaggle/input/vqa-mscoco/labelencoder.pkl', 'rb'))
labelencoder_3816 = pickle.load(open('/kaggle/working/labelencoder_3816.pkl', 'rb'))

data_df['class_label'] = labelencoder_3816.transform(list(data_df['processed_annotations']))
print("Number of Class Labels:",len(labelencoder_3816.classes_))

data_df = data_df.drop(['question_id', 'question_type','answer_type','Unnamed: 0'], axis=1)
print("Number of datapoints of final dataset:",len(data_df))
# execute thee cell once
data_df.to_csv("/kaggle/working/mscoco_train2014_preprocessed_k3816.csv",index=False)
data_df.head(3)

Number of Class Labels: 3816
Number of datapoints of final dataset: 417568


,image_id,answers,processed_questions,processed_annotations,class_label
3,train2014/COCO_train2014_000000458752.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes...",is this man a professional baseball player,yes,3801
8,train2014/COCO_train2014_000000524291.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",is the dog waiting,yes,3801
10,train2014/COCO_train2014_000000393221.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",is the sky blue,yes,3801


In [13]:
X = data_df[['image_id','processed_questions','answers']]
y = data_df['class_label']
print('X.shape:',X.shape)
print('y.shape:',y.shape)

data_df.groupby(by='class_label').count()

X.shape: (417568, 3)
y.shape: (417568,)


,image_id,answers,processed_questions,processed_annotations
class_label,,,,
0,4977,4977,4977,4977
1,7,7,7,7
2,6,6,6,6
3,12541,12541,12541,12541
4,11,11,11,11
...,...,...,...,...
3811,132,132,132,132
3812,7,7,7,7
3813,8,8,8,8


### 1.2 Take sampled 200000 datapoints

In [14]:
data_df_k3816 = pd.read_csv("/kaggle/working/mscoco_train2014_preprocessed_k3816.csv")
data_df_k3816.head(3)

,image_id,answers,processed_questions,processed_annotations,class_label
0,train2014/COCO_train2014_000000458752.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes...",is this man a professional baseball player,yes,3801
1,train2014/COCO_train2014_000000524291.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",is the dog waiting,yes,3801
2,train2014/COCO_train2014_000000393221.jpg,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",is the sky blue,yes,3801


In [15]:
# to get 400k datapoints from dataset with same %count as in dataset

X = data_df_k3816[['image_id','processed_questions','processed_annotations','answers','class_label']]
y = data_df_k3816['class_label']
print('X.shape:',X.shape)
print('y.shape:',y.shape)

X_, X_200K, y_, y_200K = train_test_split(X, y, test_size=0.75, stratify=y, random_state=42)
print('X_200K.shape:',X_200K.shape)
print('y_200K.shape:',y_200K.shape)

#execute this cell once
X_200K.to_csv("/kaggle/working/mscoco_train2014_preprocessed_k3816_200k.csv",index=False)

X.shape: (417568, 5)
y.shape: (417568,)
X_200K.shape: (313176, 5)
y_200K.shape: (313176,)


In [16]:
data_df_200k = pd.read_csv("/kaggle/working/mscoco_train2014_preprocessed_k3816_200k.csv")

X = data_df_200k[['image_id','processed_questions', 'answers']]
y = data_df_200k['class_label']
print('X.shape:',X.shape)
print('y.shape:',y.shape)

X.shape: (313176, 3)
y.shape: (313176,)


In [17]:
data_df_200k.head(3)

,image_id,processed_questions,processed_annotations,answers,class_label
0,train2014/COCO_train2014_000000171943.jpg,did they get married,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",3801
1,train2014/COCO_train2014_000000306913.jpg,are the men wearing shirts,yes,"['yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes...",3801
2,train2014/COCO_train2014_000000126707.jpg,what are the men doing,reading,"['reading', 'reading', 'reading', 'reading', '...",2763


In [18]:
# execute this cell once
# perform train validation & test split on the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.075, stratify=y, random_state=42) 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.075, stratify=y_train, random_state=42) 

pickle.dump((X_train,y_train),open('/kaggle/working/train_200k.pkl','wb'))
pickle.dump((X_val,y_val),open('/kaggle/working/val_200k.pkl','wb'))
pickle.dump((X_test,y_test),open('/kaggle/working/test_200k.pkl','wb'))

In [19]:
X_train,y_train = pickle.load(open('/kaggle/working/train_200k.pkl', 'rb'))
X_val,y_val = pickle.load(open('/kaggle/working/val_200k.pkl', 'rb'))
X_test,y_test = pickle.load(open('/kaggle/working/test_200k.pkl', 'rb'))

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

# Convert a class vector y_train and y_test to binary class matrix
Y_train = to_categorical(y_train, 3816) 
Y_val = to_categorical(y_val, 3816)
Y_test = to_categorical(y_test, 3816)

(267960, 3) (267960,)
(23489, 3) (23489,)
(21727, 3) (21727,)


### 1.3 Text Vectorization

In [20]:
# Train Validation & Test Text vectorization
t = Tokenizer(filters='')
# updates the internal vocabulary based on a list of texts.
t.fit_on_texts(list(X_train['processed_questions']))
# Tokenizer that holds a dictionary mapping each word to an integer index. 
# The size of your vocabulary is determined by the number of unique words in your dataset plus one (for the zero index not assigned to any word).
vocab_size = len(t.word_index) + 1

#execute this cell once
# pickle.dump((t),open('/kaggle/working/tokenizer_50k.pkl','wb'))

pickle.dump((t),open('/kaggle/working/tokenizer_200k.pkl','wb')) # REMOVE COMMENT

# Converts each text in the provided list to a sequence of integers, using the previously built vocabulary. Unknown words (not in the vocabulary) are skipped.
train_sequences = t.texts_to_sequences(list(X_train['processed_questions']))
# This ensures that all sequences in a list have the same length by padding them with zeros (if they are shorter than the maximum length) or truncating them (if they are longer). 
# In this case, you're using a maximum length (maxlen) of 22 and padding sequences at the end (padding='post').
train_padded_docs = pad_sequences(train_sequences, maxlen=22, padding='post')

val_sequences = t.texts_to_sequences(list(X_val['processed_questions']))
val_padded_docs = pad_sequences(val_sequences, maxlen=22, padding='post')

test_sequences = t.texts_to_sequences(list(X_test['processed_questions']))
test_padded_docs = pad_sequences(test_sequences, maxlen=22, padding='post')

In [21]:
print(vocab_size)

11767


In [22]:
import numpy as np

# Initialize an empty dictionary to store GloVe vectors
glovevector = {}

# Path to the GloVe vectors text file
file_path = '/kaggle/input/vqa-mscoco/glove.6B/glove.6B.300d.txt'  # Update with the actual file path

# Open the file and read its contents line by line
with open(file_path, 'r') as file:
    for line in file:
        # Split each line into word and vector components
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype=np.float32)
        # Store the word and its vector in the dictionary
        glovevector[word] = vector

print("Loaded GloVe vectors:", len(glovevector))
print(type(glovevector))


Loaded GloVe vectors: 400001
<class 'dict'>


In [23]:
print('Type:',type(glovevector))
print('Size:',len(glovevector))
print('Dim:',glovevector['the'].shape)

Dim: (300,)


In [24]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = glovevector.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(11767, 300)


### 1.4 Data Generator Function

In [26]:
aug1 = iaa.Fliplr(0.5)  # Horizontal flip with a probability of 0.5
aug2 = iaa.GaussianBlur(sigma=(0.0, 1.0))  # Apply Gaussian blur with sigma ranging from 0.0 to 1.0
aug3 = iaa.Multiply((0.5, 1.5), per_channel=0.5)  # Multiply pixel values by random values ranging from 0.5 to 1.5
aug4 = iaa.Add((-40, 40), per_channel=0.5)  # Add random values ranging from -40 to 40 to pixel values
aug5 = iaa.Sometimes(0.5, iaa.CropAndPad(percent=(-0.1, 0.1)))  # Crop and pad images with a probability of 0.5

In [27]:
# # https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

class CustomDataGen_aug(tf.keras.utils.Sequence):
    
    # I give the question, image, label as input to the data generator, set shuffle = True, indexes - after each epoch keeps track of the data
    
    def __init__(self, X_que, X_img, y,
                 batch_size,
                 shuffle=True):
        
        self.X_que = X_que
        self.X_img = X_img
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(y))
        
    # if shuffling is enabled, each time the model is executed the model will see different data order, which prevenets from overfitting
        
    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
            
    # input1 = textual questions - converts questions to sequences of integers based on tokenizer's vocabulary and then pads these questions to a fixed length to ensure consistent input size
    
    def __get_input1(self, que):
    
        que_arr = (pad_sequences(t.texts_to_sequences([que]), maxlen=22, padding='post'))[0]
        return que_arr
    
    # input2 = process the images - random augmentation - normalize the image size

    def __get_input2(self, path):
        # Load the image from the specified path
        img = cv2.imread(path)
        if img is None:
            # If the image is not read correctly, handle the error, for example, by logging and using a placeholder image
            print(f"Warning: Image not read from path {path}")
            img = np.zeros((448, 448, 3))  # Placeholder for an empty image, adjust size as needed
        else:
            # Resize the image to a fixed size (e.g., 224x224, which is common for CNNs)
            img = cv2.resize(img, (448, 448))

            # Apply random augmentation based on a uniform distribution
            a = np.random.uniform()
            if a < 0.20:
                img = aug1.augment_image(img)
            elif a < 0.40:
                img = aug2.augment_image(img)
            elif a < 0.60:
                img = aug3.augment_image(img)
            elif a < 0.80:
                img = aug4.augment_image(img)
            elif a < 1.00:
                img = aug5.augment_image(img)
            # If 'a' is between 0.75 and 1, no augmentation is applied

        # Normalize the image by converting pixel values to the range [0, 1]
        img = np.array(img, dtype=np.float32) / 255.0

        return img
    
    # converts the numerical class labels into one-hot encoded vectors, making suitable for categorical classification
    
    def __get_output(self, label):
        return tf.keras.utils.to_categorical(label, num_classes=3816)
    
    # for a batch index - it extracts the questions, images from (input1 ,input2) and prepares the corresponding labels, returns the tuple (input1,input2,label)
    
    def __getitem__(self, index):

        batch_x0 = self.X_que[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x1 = self.X_img[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        X0_batch = np.asarray([self.__get_input1(que) for que in batch_x0])
        X1_batch = np.asarray([self.__get_input2("/kaggle/input/vqa-mscoco/train2014/"+path) for path in batch_x1])
#         print([np.array(self.__get_input2("/kaggle/input/vqa-mscoco/train2014/" + path)).shape for path in batch_x1])
        y_batch = np.asarray([self.__get_output(c) for c in batch_y])
       
        return tuple([X0_batch, X1_batch]), y_batch
    
    # number of batches per epoch based on the dataset size and batch size
    
    def __len__(self):
        return len(self.indexes) // self.batch_size


class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, X_que, X_img, y,
                 batch_size,
                 shuffle=True):
        
        self.X_que = X_que
        self.X_img = X_img
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(y))

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
    
    def __get_input1(self, que):
    
        que_arr = (pad_sequences(t.texts_to_sequences([que]), maxlen=22, padding='post'))[0]
        return que_arr

    def __get_input2(self, path):
        # Load the image from the specified path
        img = cv2.imread(path)
        if img is None:
            # If the image is not read correctly, handle the error, for example, by logging and using a placeholder image
            print(f"Warning: Image not read from path {path}")
            img = np.zeros((448, 448, 3))  # Placeholder for an empty image, adjust size as needed
        else:
            # Resize the image to a fixed size (e.g., 224x224, which is common for CNNs)
            img = cv2.resize(img, (448, 448))
        # Normalize the image by converting pixel values to the range [0, 1]
        img = np.array(img, dtype=np.float32) / 255.0

        return img
    
    def __get_output(self, label):
        return tf.keras.utils.to_categorical(label, num_classes=3816)
    
    def __getitem__(self, index):

        batch_x0 = self.X_que[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x1 = self.X_img[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        X0_batch = np.asarray([self.__get_input1(que) for que in batch_x0])
        X1_batch = np.asarray([self.__get_input2("/kaggle/input/vqa-mscoco/train2014/"+path) for path in batch_x1])
        y_batch = np.asarray([self.__get_output(c) for c in batch_y])
       
        return tuple([X0_batch, X1_batch]), y_batch
    
    def __len__(self):
        return len(self.indexes) // self.batch_size

batch_siz = 256
traingen = CustomDataGen_aug(list(X_train['processed_questions']),list(X_train['image_id']),list(y_train),batch_size=batch_siz)
valgen = CustomDataGen(list(X_val['processed_questions']),list(X_val['image_id']),list(y_val),batch_size=batch_siz)

In [28]:
# For __get_input1 (text questions processing):
#sample_question = X_train['processed_questions'][0]  # Replace with any question from your dataset
sample_question = X_train['processed_questions'].iloc[0]  # Aks modified
processed_question = traingen._CustomDataGen_aug__get_input1(sample_question)
print(sample_question)
print(processed_question)


is this a game
[  2   5   8 115   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [29]:
# For __get_input2 (image processing):
sample_image_path = "/kaggle/input/vqa-mscoco/train2014/" + X_train['image_id'].iloc[0]  # Replace with any image path from your dataset
processed_image = traingen._CustomDataGen_aug__get_input2(sample_image_path)
print(sample_image_path)
print(processed_image)


/kaggle/input/vqa-mscoco/train2014/train2014/COCO_train2014_000000553894.jpg
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [30]:
y_train

151920    2252
142512     735
136239    3801
245587    2252
93460     2724
          ... 
91767     2252
195630    1580
166979     105
21031     2252
257804    3702
Name: class_label, Length: 267960, dtype: int64

In [31]:
# For __get_output (labels processing):
sample_label = y_train[10]  # Replace with any label from your dataset
one_hot_label = traingen._CustomDataGen_aug__get_output(sample_label)
print(sample_label)
print(one_hot_label)

3271
[0. 0. 0. ... 0. 0. 0.]


### 1.5 Create features for Image Test Data

In [ ]:
def Dataset(colab_path, que, image_id, y, shape):
    img = cv2.imread(os.path.join(colab_path,image_id))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Resize image to the specified shape
    img = cv2.resize(img, (shape, shape), interpolation=cv2.INTER_NEAREST)
    image_vector = (img/255.0).astype(np.float32)
    
    que_vector = (pad_sequences(t.texts_to_sequences([que]), maxlen=22, padding='post'))[0]
    que_vector = np.asarray(que_vector).astype(np.float32)
    
    y = tf.one_hot(tf.cast(int(y), tf.uint8), 3816)
    y = np.asarray(y).astype(np.float32)
    return que_vector, image_vector, y


test_image = []; test_que = []; Y_test = []
for i in tqdm(range(len(X_test))):
    que, image, y = Dataset("/kaggle/input/vqa-mscoco/train2014/",list(X_test['processed_questions'])[i],list(X_test['image_id'])[i],list(y_test)[i],448)
    test_image.append(image)
    test_que.append(que)
    Y_test.append(y)
test_image = np.asarray(test_image).astype(np.float32)
test_que = np.asarray(test_que).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.float32)

 53%|█████▎    | 11424/21727 [05:54<3:43:36,  1.30s/it]

In [ ]:
# excute this cell once
# pickle.dump((test_image),open('/kaggle/working/test_image_50k_0711.pkl','wb'))
pickle.dump((test_image),open('/kaggle/working/test_image_400k.pkl','wb'))

In [ ]:
test_image= pickle.load(open('/kaggle/working/test_image_400k.pkl', 'rb'))
# test_image= pickle.load(open('/kaggle/input/vqa-mscoco/test_image_400k.pkl', 'rb'))
test_que = test_padded_docs


### 1.6 Model Metric for Prediction (As per Research Paper)

In [ ]:
#  Research Paper Link: https://arxiv.org/pdf/1505.00468.pdf

labelencoder_3816 = pickle.load(open('/kaggle/working/labelencoder_3816.pkl', 'rb'))
# labelencoder_3816 = pickle.load(open('/kaggle/input/vqa-mscoco/labelencoder_3816.pkl', 'rb'))

def accuracy_metric(X,Y,encoded_features,model):
    predicted_Y = model.predict(encoded_features,verbose=0)
    predicted_class = tf.argmax(predicted_Y, axis=1, output_type=tf.int32)
    predicted_ans = labelencoder_3816.inverse_transform(predicted_class)
    acc_val_lst = []
    for i in tqdm(range(len(Y))):
        acc_val = 0.0
        temp = 0
        
        for actual_ans in (list(X['answers'])[i]).split(","):
            if actual_ans == predicted_ans[i]:
                temp += 1
      
    if temp >= 3:
        acc_val = 1
    else:
        acc_val = float(temp)/3
  
    acc_val_lst.append(acc_val)

    
    return (sum(acc_val_lst)/len(Y))*100

### 1.7 LSTM + VGG19 Model

In [ ]:
pre_trained_model = tf.keras.applications.VGG19(input_shape=(448, 448, 3), include_top=True, weights="imagenet", pooling='avg')
for layer in pre_trained_model.layers:
    layer.trainable = False 

regularizer = tf.keras.regularizers.l2(0.01)

for layer in pre_trained_model.layers:
    for attr in ['kernel_regularizer']:
        if hasattr(layer, attr):
            setattr(layer, attr, regularizer)

vgg19_fc2_output = (pre_trained_model.get_layer('fc2')).output
img = Dense(units=1024,activation='relu',kernel_initializer='he_normal')(vgg19_fc2_output)
img = Dropout(0.2)(img)

input_layer_que = Input(shape=(22,))
# Initialize the embedding layer separately
embedding_layer = Embedding(vocab_size, 300, trainable=True)

# Pass the input to the embedding layer
embedding = embedding_layer(input_layer_que)


lstm1 = LSTM(64, return_sequences=True)(embedding)
dropout1 = Dropout(0.5)(lstm1) 
lstm2 = LSTM(64)(dropout1) 
dropout2 = Dropout(0.5)(lstm2) 
que = Dense(units=1024,activation='relu',kernel_initializer='he_normal')(dropout2)

@tf.keras.utils.register_keras_serializable() # Line added
class MultiplyLayer(Layer):
    def __init__(self, **kwargs):
        super(MultiplyLayer, self).__init__(**kwargs)
        
    def call(self, inputs):
        que, img = inputs
        return tf.math.multiply(que, img)

# Define the pointwise multiplication layer
pointwise_mul = MultiplyLayer()([que, img])

# Define the output layer
# output = Dense(units=1000, activation='softmax', kernel_initializer="glorot_uniform")(pointwise_mul)
output = Dense(units=3816, activation='softmax', kernel_initializer="glorot_uniform")(pointwise_mul)

# Define the model with input and output layers
model_lstm_vgg19 = Model(inputs=[input_layer_que, pre_trained_model.input], outputs=output)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_lstm_vgg19.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model_lstm_vgg19.summary()

In [ ]:
plot_model(model_lstm_vgg19, to_file='model_plot1.png', show_shapes=True, show_layer_names=True)

In [ ]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
import keras
print("Standalone Keras Version:", keras.__version__)

In [ ]:
checkpoint_filepath="/kaggle/working/weights-{epoch:02d}-{val_accuracy:.4f}.keras"
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

In [ ]:
history = model_lstm_vgg19.fit(traingen, batch_size=256, epochs=5, verbose=1, validation_data=valgen, callbacks=[model_checkpoint_callback])

In [ ]:
#model_lstm_vgg19.load_weights("/kaggle/working/weights-02-0.2922.hdf5")
#model_lstm_vgg19.load_weights("/kaggle/input/vqa-mscoco/weights-01-0.2714.hdf5")
model_lstm_vgg19.load_weights("/kaggle/input/vqa-mscoco/weights-01-0.3131.keras")

In [ ]:
model_lstm_vgg19.load_weights("/kaggle/working/weights...")


In [ ]:
history_1 = model_lstm_vgg19.fit(traingen, batch_size=256, epochs=1, verbose=1, validation_data=valgen, callbacks=[model_checkpoint_callback])

In [ ]:
#model_lstm_vgg19.load_weights("/kaggle/working/weights-01-0.3045.keras")
# model_lstm_vgg19.load_weights("/kaggle/input/vqa-mscoco/weights...")

In [ ]:
history_2 = model_lstm_vgg19.fit(traingen, batch_size=128, epochs=1, verbose=1, validation_data=valgen, callbacks=[model_checkpoint_callback])

In [ ]:
history_3 = model_lstm_vgg19.fit(traingen, batch_size=256, epochs=3, verbose=1, validation_data=valgen, callbacks=[model_checkpoint_callback])

In [ ]:
history_4 = model_lstm_vgg19.fit(traingen, batch_size=256, epochs=5, verbose=1, validation_data=valgen, callbacks=[model_checkpoint_callback])
add Codeadd Markdown

In [ ]:
# history = model_lstm_vgg19.fit(traingen,batch_size=128,epochs=10,verbose=1,validation_data=valgen,callbacks=[model_checkpoint_callback])

In [ ]:
# history1 = model_lstm_vgg19.fit(traingen,batch_size=128,epochs=25,verbose=1,validation_data=valgen,callbacks=[model_checkpoint_callback])

In [ ]:
# model_lstm_vgg19.load_weights("/content/drive/MyDrive/Applied AI/CS2/model_1011_2/weights-12-0.3628.hdf5")

In [ ]:
# history = model_lstm_vgg19.fit(traingen,batch_size=128,epochs=12,verbose=1,validation_data=valgen,callbacks=[model_checkpoint_callback])

In [ ]:
# model_lstm_vgg19.load_weights("/content/drive/MyDrive/Applied AI/CS2/model_1011_2/weights-01-0.3634.hdf5")

In [ ]:
# history = model_lstm_vgg19.fit(traingen,batch_size=128,epochs=12,verbose=1,validation_data=valgen,callbacks=[model_checkpoint_callback])

In [ ]:
# model_lstm_vgg19.load_weights("/content/drive/MyDrive/Applied AI/CS2/model_1011_2/weights-09-0.3650.hdf5")

In [ ]:
# history = model_lstm_vgg19.fit(traingen,batch_size=128,epochs=4,verbose=1,validation_data=valgen,callbacks=[model_checkpoint_callback])

In [ ]:
history_dict = history.history
history_dict2 = history1.history1
# Convert to DataFrame
history_df = pd.DataFrame(history_dict)

# Save DataFrame to CSV
history_df.to_csv('history.csv', index=False)

In [ ]:
df = pd.read_csv("/kaggle/working/training_history.csv")

# Plot training & validation loss values
plt.figure(figsize=(15, 4)); plt.subplot(121)
plt.plot(list(df['loss'])); plt.plot(list(df['val_loss']))
plt.title('Train & Validation loss'); plt.ylabel('loss'); plt.xlabel('Epoch'); plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation Accuracy values
plt.subplot(122)
plt.plot(list(df['accuracy'])); plt.plot(list(df['val_accuracy']))
plt.title('Train & Validation accuracy'); plt.ylabel('accuracy'); plt.xlabel('Epoch'); plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
Test_accuracy = accuracy_metric(X_test,Y_test,[test_que,test_image],model_lstm_vgg19)
print("\nTest accuracy of LSTM+VGG19 Model 50k Datapoints:",Test_accuracy)  # 41.42

In [ ]:
model_lstm_vgg19.save('/kaggle/working/model.keras')

In [ ]:
model_lstm_vgg19 = tf.keras.models.load_model('/kaggle/working/model.keras')

In [ ]:
# pickle.dump(model_lstm_vgg19, open("/content/drive/MyDrive/Applied AI/CS2/model/final_model.pkl", "wb"))
pickle.dump(model_lstm_vgg19, open("/kaggle/working/final_model.pkl", "wb"))

In [ ]:
test_image_path = list(X_test['image_id'])
test_question = list(X_test['processed_questions'])
test_answer = list(y_test)

colab_path ="/kaggle/input/vqa-mscoco/train2014/"
# Plot test images, questions, actual answers and predicted answers
fig = plt.figure(figsize=(24, 10))
for a,i in enumerate(list(np.arange(10))):
  fig.add_subplot(2, 5, a+1)
  img = cv2.imread(colab_path+test_image_path[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img); plt.axis('off')
  plt.title(f'Que: {test_question[i]} \nActual Ans: {labelencoder_3816.inverse_transform(test_answer)[i]} \nPredicted Ans: {labelencoder_3816.inverse_transform([np.argmax(model_lstm_vgg19.predict([np.array([test_que[i]]),np.array([test_image[i]])],verbose=0))])[0]}')

In [ ]:
test_image_path = list(X_test['image_id'])
test_question = list(X_test['processed_questions'])
test_answer = list(y_test)

# Plot test images, questions, actual answers and predicted answers
fig = plt.figure(figsize=(24, 10))
for a,i in enumerate(list(np.arange(5,15))):
  fig.add_subplot(2, 5, a+1)
  img = cv2.imread(colab_path+test_image_path[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img); plt.axis('off')
  plt.title(f'Que: {test_question[i]} \nActual Ans: {labelencoder_3816.inverse_transform(test_answer)[i]} \nPredicted Ans: {labelencoder_3816.inverse_transform([np.argmax(model_lstm_vgg19.predict([np.array([test_que[i]]),np.array([test_image[i]])],verbose=0))])[0]}')

In [ ]:
test_image_path = list(X_test['image_id'])
test_question = list(X_test['processed_questions'])
test_answer = list(y_test)

# Plot test images, questions, actual answers and predicted answers
fig = plt.figure(figsize=(24, 10))
for a,i in enumerate(list(np.arange(20,30))):
  fig.add_subplot(2, 5, a+1)
  img = cv2.imread(colab_path+test_image_path[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img); plt.axis('off')
  plt.title(f'Que: {test_question[i]} \nActual Ans: {labelencoder_3816.inverse_transform(test_answer)[i]} \nPredicted Ans: {labelencoder_3816.inverse_transform([np.argmax(model_lstm_vgg19.predict([np.array([test_que[i]]),np.array([test_image[i]])],verbose=0))])[0]}')

## 2. Results

In [ ]:
from prettytable import PrettyTable
x2 = PrettyTable(["Model", "Datapoints", "Class Labels", "Sampled Datapoints", "Epochs", "Train Accuracy", "Validation Accuracy", "Test Accuracy"])
x2.add_row(["LSTM+VGG19", "380554", "1000", "50000", "50", round(Train_accuracy,3), round(Val_accuracy,3), round(Test_accuracy,3)])
print(x2)